# Supplemental Material

Lagged Predictions of Next Week Alcohol Use for Precision Mental Health Support

Kendra Wyant, Gaylen E. Fronk, Jiachen Yu, and John J. Curtin  
June 11, 2025

This file contains the supplemental materials for *Lagged Predictions of Next Week Alcohol Use for Precision Mental Health Support*. It includes a transparency report and all supplemental figures and tables. Additional materials are made available on our study’s OSF page (<https://osf.io/xta67/>).

------------------------------------------------------------------------

In [ ]:
#| echo: false
#| code-fold: true

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
suppressPackageStartupMessages(library(tidyposterior))
suppressPackageStartupMessages(library(kableExtra))
library(patchwork)


theme_set(theme_classic())

options(knitr.kable.NA = '')

path_models_lag <- format_path(str_c("risk/models/lag"))
path_processed <- format_path(str_c("risk/data_processed/lag"))

In [ ]:
#| echo: false
#| code-fold: true

pp_tidy <- read_csv(here::here(path_models_lag, "posteriors.csv"), 
                                 show_col_types = FALSE) 

ci <- read_csv(here::here(path_models_lag, "pp_perf_tibble.csv"), 
                                 show_col_types = FALSE) |> 
  mutate(model = factor(model, levels = c("0 lag", "24 lag", "72 lag", "168 lag", "336 lag"),
                        labels = c("No lag", "1 day", "3 days", "1 week", "2 weeks")))

pp_dem <- read_csv(here::here(path_models_lag, "pp_dem_all.csv"), 
                                 show_col_types = FALSE)

pp_dem_contrast <- read_csv(here::here(path_models_lag, "pp_dem_contrast_all.csv"), 
                                 show_col_types = FALSE) |> 
  mutate(lag = factor(lag, levels = c("0", "24", "72", "168", "336"),
                        labels = c("No lag", "1 day", "3 days", "1 week", "2 weeks")))

labels <- read_csv(here::here(path_processed, "labels_1day_0lag.csv"),
                   show_col_types = FALSE)

dem_tbl <- read_csv(here::here(path_processed, "dem_tibble.csv"),
                   show_col_types = FALSE)

labels_dem <- labels |> 
  left_join(dem_tbl, by = "subid")

## Transparency Report 1.0 (full, 36 items; Aczel et al., 2019)

**Manuscript Title:** Lagged Predictions of Next Week Alcohol Use for Precision Mental Health Support  
**Authors:** Kendra Wyant, Gaylen E. Fronk, Jiachen Yu, and John J. Curtin  
**Corresponding author’s email address:** jjcurtin@wisc.edu  
**Link to Project Repository:** <https://osf.io/xta67/>

### Preregistration Section

-   Prior to analyzing the complete data set, a time-stamped preregistration was posted in an independent, third-party registry for the data analysis plan: Yes

Comments about your Preregistration: We pre-registered our data analytic strategy on OSF.

### Methods Section

The manuscript fully describes…

-   the rationale for the sample size used (e.g., an a priori power analysis): Yes  
-   how participants were recruited: Yes  
-   how participants were selected (e.g., eligibility criteria): Yes  
-   what compensation was offered for participation: Yes  
-   how participant dropout was handled (e.g., replaced, omitted, etc): Yes  
-   how participants were assigned to conditions: N/A. There are no conditions.  
-   how stimulus materials were randomized: N/A.  
-   whether (and, if so, how) participants, experimenters, and data-analysts were kept naive to potentially biasing information: N/A. This is an observations study that does not include analysis of group or manipulations. There were no study conditions to blind.  
-   the study design, procedures, and materials to allow independent replication: Yes  
-   the measures of interest (e.g., friendliness): Yes  
-   all operationalizations for the measures of interest (e.g., a questionnaire measuring friendliness): Yes

### Results and Discussion Section

The manuscript…

-   distinguishes explicitly between “confirmatory” (i.e., prespecified) and “exploratory” (i.e., not prespecified) analyses: All analyses were pre-registered.
-   describes how violations of statistical assumptions were handled: No  
-   justifies all statistical choices (e.g., including or excluding covariates; applying or not applying transformations; use of multi-level models vs. ANOVA): Yes  
-   reports the sample size for each cell of the design: Yes  
-   reports how incomplete or missing data were handled: Yes  
-   presents protocols for data preprocessing (e.g., cleaning, discarding of cases and items, normalizing, smoothing, artifact correction): Yes

### Data, Code, and Materials Availability Section

The following have been made publicly available…

-   the (processed) data, on which the analyses of the manuscript were based: Yes  
-   all code and software (that is not copyright protected): Yes  
-   all instructions, stimuli, and test materials (that are not copyright protected): Yes  
-   Are the data properly archived (i.e., would a graduate student with relevant background knowledge be able to identify each variable and reproduce the analysis): Yes  
-   The manuscript includes a statement concerning the availability and location of all research items, including data, materials, and code relevant to the study: Yes

## Supplemental Figures

### Figure S1: Raw Predicted Probabilities

In [ ]:
#| code-fold: true

preds_0 <- read_rds(here::here(path_models_lag, 
                               "final_preds_kfold_1_x_5_1day_0_v3_strat_lh_final.rds")) 
preds_24 <- read_rds(here::here(path_models_lag, 
                               "final_preds_kfold_1_x_5_1day_24_v3_strat_lh_final.rds")) 
preds_72 <- read_rds(here::here(path_models_lag, 
                               "final_preds_kfold_1_x_5_1day_72_v3_strat_lh_final.rds")) 
preds_168 <- read_rds(here::here(path_models_lag, 
                               "final_preds_kfold_1_x_5_1day_168_v3_strat_lh_final.rds")) 
preds_336 <- read_rds(here::here(path_models_lag, 
                               "final_preds_kfold_1_x_5_1day_336_v3_strat_lh_final.rds")) 

In [ ]:
#| code-fold: true

preds_all <- preds_0 |> 
  mutate(model = "No lag") |> 
  bind_rows(preds_24 |> 
              mutate(model = "1 day")) |> 
  bind_rows(preds_72 |> 
              mutate(model = "3 days")) |> 
  bind_rows(preds_168 |> 
              mutate(model = "1 week")) |> 
  bind_rows(preds_336 |> 
              mutate(model = "2 weeks")) |> 
  mutate(model = factor(model, levels = c("No lag", "1 day", "3 days", "1 week", "2 weeks")))

Our models produced variation in predicted probabilities, with probabilities spanning nearly the entire 0 - 1 range. The 2-week lagged model had fewer high raw probabilities (all probabilities were \< .80). Histograms of the raw predicted probabilities for all 5 models are presented below.

In [ ]:
#| fig-cap: "Histograms of raw probabilities of our best model configurations for each classification model (no lag, 1-day, 3-day, 1-week and 2-week lags)."
#| fig-width: 8
#| fig-height: 5

preds_all |> 
  ggplot(aes(x = prob_raw)) +
  geom_histogram(color = "black", fill = "light grey", binwidth = .10,
                 boundary = 0) +
  facet_wrap(~model) +
  labs(x = "Raw probabilities from final model") +
  scale_x_continuous(breaks = seq(0, 1, by = 0.2))

### Figure S2: Calibration Plots for all Models

In [ ]:
#| code-fold: true

bin_width = 0.10

preds_all_raw <- preds_0 |> 
  mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
         lapse = if_else(label == "Lapse", 1, 0)) |> 
  mutate(model = "No lag") |> 
  bind_rows(preds_24 |> 
              mutate(bins = cut(prob_raw, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "1 day")) |> 
  bind_rows(preds_72 |> 
              mutate(bins = cut(prob_raw, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "3 days")) |> 
  bind_rows(preds_168 |> 
              mutate(bins = cut(prob_raw, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "1 week")) |> 
  bind_rows(preds_336 |> 
              mutate(bins = cut(prob_raw, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "2 weeks")) |> 
  mutate(model = factor(model, levels = c("No lag", "1 day", "3 days", "1 week", "2 weeks")))

preds_all_beta <- preds_0 |> 
  mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
         lapse = if_else(label == "Lapse", 1, 0)) |> 
  mutate(model = "No lag") |> 
  bind_rows(preds_24 |> 
              mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "1 day")) |> 
  bind_rows(preds_72 |> 
              mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "3 days")) |> 
  bind_rows(preds_168 |> 
              mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "1 week")) |> 
  bind_rows(preds_336 |> 
              mutate(bins = cut(prob_beta, breaks = seq(0, 1, bin_width)), 
                     lapse = if_else(label == "Lapse", 1, 0)) |> 
              mutate(model = "2 weeks")) |> 
  mutate(model = factor(model, levels = c("No lag", "1 day", "3 days", "1 week", "2 weeks")))

The raw probabilities produced by our final models were not well calibrated. Brier scores for the no lag model (.071) and the 2-week lag model (.077) were roughly equivalent to the base rate of lapses (.076). Below we present the calibration plots of our raw predicted lapse probabilities for each model.

In [ ]:
#| fig-cap: "Calibration plots of raw predicted lapse probabilities for each model (no lag, 1-day, 3-day, 1-week, and 2-week lag). Predicted lapse probability (x-axis) is binned in deciles. Observed lapse probabilities (y-axis) represent the proportion of true lapses observed in each bin. The dashed line represents a perfectly calibrated model. "
#| fig-width: 8
#| fig-height: 5

preds_all_raw |> 
  group_by(bins, model)  |> 
  summarize(mean_lapse = mean(lapse),
            .groups = "drop") |> 
  mutate(bins = as.numeric(bins),
         midpoints = bin_width/2 + bin_width * (bins - 1))  |> 
  ggplot(data = _, aes(x = midpoints, y = mean_lapse)) +
  geom_abline(slope = 1, intercept = 0, linetype = "dotted") +
  geom_line() +
  geom_point() +
  facet_wrap(~ model) +
  labs(x = "Predicted Lapse Probability (bin mid-point)",
       y = "Observed Lapse Probability",
       title = "Raw Probabilities") +
  scale_x_continuous(breaks = seq(0, 1, bin_width),
                     limits = c(0, 1)) +
  scale_y_continuous(breaks = seq(0, 1, bin_width),
                     limits = c(0, 1)) 

Beta-calibration was selected as the best calibration method (i.e., it had the lowest average brier score across the no lag and 2-week lag models). Brier scores for the no lag model (.043) and the 2-week model (.061) indicated improved calibration with the beta transformation. Notably, there was a 40% reduction in error for no lag beta-calibrated probabilities compared to the no lag raw probabilities (there was a corresponding 21% reduction in error for the 2-week lag model). Below we present calibration plots for the beta-calibrated probabilities for all 5 models.

In [ ]:
#| fig-cap: "Calibration plots of beta-calibrated predicted lapse probabilities for each model (no lag, 1-day, 3-day, 1-week, and 2-week lag). Predicted lapse probability (x-axis) is binned in deciles. Observed lapse probabilities (y-axis) represent the proportion of true lapses observed in each bin. The dashed line represents a perfectly calibrated model. "
#| fig-width: 8
#| fig-height: 5

preds_all_beta |> 
  group_by(bins, model)  |> 
  summarize(mean_lapse = mean(lapse),
            .groups = "drop") |> 
  mutate(bins = as.numeric(bins),
         midpoints = bin_width/2 + bin_width * (bins - 1))  |> 
  ggplot(data = _, aes(x = midpoints, y = mean_lapse)) +
  geom_abline(slope = 1, intercept = 0, linetype = "dotted") +
  geom_line() +
  geom_point() +
  facet_wrap(~ model) +
  labs(x = "Predicted Lapse Probability (bin mid-point)",
       y = "Observed Lapse Probability",
       title = "Beta Calibration") +
  scale_x_continuous(breaks = seq(0, 1, bin_width),
                     limits = c(0, 1)) +
  scale_y_continuous(breaks = seq(0, 1, bin_width),
                     limits = c(0, 1)) 